# تمرین کامپیوتری دوم - قسمت دوم(ژنتیک)
#  نیما مدیرکیاسرایی 810198471

## هدف بازی :  
در این پروژه با استفاده از الگوریتم های ژنتیک که روش های کلی Crossover و Mutation در آن وجود دارد ، مساله ی معادله برابری (Equation Problem) را پیاده سازی می کنیم.
## پیاده سازی مساله :
### مشخص کردن مفاهیم اولیه و تولید جمعیت اولیه :
در این الگوریتم هر ژن در واقع هر operand یا هر operator می باشد و مجموعه تعدادی از این ژن ها کنار هم (که در این مساله 21 عدد است) یک کروموزوم تشکیل می دهد.
تولید جمعیت اولیه بصورت رندوم در متد makeFirstPopulation انجام شده است به طوری که به اندازه جمعیت که 100 تعریف شده است کروموزوم به شکل تصادفی تولید می کند.
###  پیاده سازی و مشخص کردن تابع معیار سازگاری :
تابع معیار سازگاری ما در متد calcFitness قرار دارد و در حقیقت تابعی که در این مساله از آن استفاده کرده ایم (1+x)/1 می باشد که x اختلاف محاسبه واقعی هر کروموزوم با عددی است که میخواهیم به آن برسیم. در حقیقت وقتی تابع را به این شکل تعریف می کنیم fitness هر کروموزوم عددی بین 0 تا 1 خواهد بود.
### پیاده سازی crossover و mutation و تولید جمعیت بعدی :
تابع crossover بر روی دو کروموزوم اعمال می شود، و آنها را ترکیب میکند تا به کروموزوم هایی از ترکیب آن دو که در حالت ایده آل بهترین ویژگی های دو ژن اولیه را دارند برسد.
تابع mutation بر روی یک کروموزوم اعمال میشود، و آن را جهش و یا تغییر میدهد؛ به امید آن که بتواند به کروموزوم بهتری دست یابد.
همچنین در این مساله عدد 20% را برای انتقال مستقیم ژن های برتر به نسل های آینده انتخاب کرده ایم. همچنین احتمال crossover را برابر 50 درصد قرار داده ایم.
متد های createCrossoverPool و mutate به ترتیب برای پیاده سازی crossover و mutate می باشند.
### ایجاد الگوریتم ژنتیک روی مسئله :


In [2]:
import random
import numpy as np
#TODO assign numbers to variables below
crossoverProbability = 0.5
carryPercentage = 0.2
populationSize = 100

class EquationBuilder:
    
    def __init__(self, operators, operands, equationLength, goalNumber):
        self.operators = operators
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber

        # Create the earliest population at the begining
        self.population = self.makeFirstPopulation()
        
    def makeFirstPopulation(self):
        #TODO create random chromosomes to build the early population, and return it
        chromosomes =[]
        for i in range(populationSize):
            temp = []
            operands_sample = random.choices(operands , k = int(self.equationLength/2)+1)
            operators_simple = random.choices(operators , k = int(self.equationLength/2))
            zipped = zip(operands_sample,operators_simple)
            for item in zipped:
                temp.append(item[0]) 
                temp.append(item[1])
            temp.append(operands_sample[-1])
            chromosomes.append(temp)
        return chromosomes 
    
    def findEquation(self):
        # Create a new generation of chromosomes, and make it better in every iteration
        while (True):
            random.shuffle(self.population)

            fitnesses = []
            for i in range(populationSize):
                #TODO calculate the fitness of each chromosome
                #TODO return chromosome if a solution is found, else save the fitness in an array
                fitness = self.calcFitness(self.population[i])
                if (fitness == 1):
                    return self.population[i]
                fitnesses.append(fitness)
            self.fitnesses = fitnesses    

            #TODO find the best chromosomes based on their fitnesses, and carry them directly to the next generation (optional)
            bestChromosome = []
            sorted_fitnesses = sorted(fitnesses)[:int(populationSize*carryPercentage)]
            for item in sorted_fitnesses:
                bestChromosome.append(self.population[fitnesses.index(item)])
            carriedChromosomes = []
            for i in range(0, int(populationSize*carryPercentage)):
                carriedChromosomes.append(bestChromosome[i]) 

            # A pool consisting of potential candidates for mating (crossover and mutation)    
            matingPool = self.createMatingPool(populationSize - int(carryPercentage * populationSize))

            # The pool consisting of chromosomes after crossover
            crossoverPool = self.createCrossoverPool(matingPool)

            # Delete the previous population
            self.population.clear()

            # Create the portion of population that is undergone crossover and mutation
            for i in range(populationSize - int(populationSize*carryPercentage)):
                self.population.append(self.mutate(crossoverPool[i]))
                
            # Add the prominent chromosomes directly to next generation
            self.population.extend(carriedChromosomes)
    
    def createMatingPool(self , size):
        #TODO make a brand new custom pool to accentuate prominent chromosomes (optional)
        #TODO create the matingPool using custom pool created in the last step and return it
        probability = []
        matingPool = []
        for item in self.fitnesses :
            probability.append(item/sum(self.fitnesses)) 
        indecies = np.random.choice(list(range(len(self.fitnesses))), size, p=probability)
        for index in indecies:
            matingPool.append(self.population[index])
        return matingPool
    
    def createCrossoverPool(self, matingPool):
        crossoverPool = []
        for i in range(len(matingPool)):
            if random.random() > crossoverProbability:
                #TODO don't perform crossover and add the chromosomes to the next generation directly to crossoverPool
                crossoverPool.append(matingPool[i])
            else:
                #TODO find 2 child chromosomes, crossover, and add the result to crossoverPool
                location = random.randint(1,self.equationLength)
                chromosome1 = random.randint(0, len(matingPool)-1)
                chromosome2 = random.randint(0, len(matingPool)-1)
                new_chromosome= matingPool[chromosome1][:location] + matingPool[chromosome2][location:]
                crossoverPool.append(new_chromosome)
        return crossoverPool
    
    def mutate(self, chromosome):
        #TODO mutate the input chromosome 
        random_index = random.randint(0, self.equationLength - 1)
        if chromosome[random_index] in self.operands:
            chromosome[random_index] = random.choice(self.operands)
        else:
            chromosome[random_index] = random.choice(self.operators)
        return chromosome

    def calcFitness(self, chromosome):
        #TODO define the fitness measure here
        temp = []
        for item in chromosome:
            temp.append(str(item))
        distance = abs(goalNumber - eval(''.join(temp)))
        return 1 / (1 + distance)


operands = [1, 2, 3, 4, 5, 6, 7, 8]
operators = ['+', '-', '*']
equationLength = 21
goalNumber = 18019

equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber)
equation = equationBuilder.findEquation()
for i in range(len(equation)):
    equation[i] = str(equation[i])
print(' '.join(equation))

8 * 6 * 8 * 1 - 5 + 6 * 7 * 7 * 2 * 6 * 5


### سوالات :
1) اگر جمعیت اولیه بسیار کم باشد تنوع در کروموزوم ها و جواب های حاصل از آن ها خیلی کم خواهد بود بنابراین مدت زمان خیلی بیشتری طول می کشد تا همگرایی رخ دهد و الگوریتم به جواب درست برسد اما اگر جمعیت بسیار زیاد باشد تعداد باری که الگوریتم باید اجرا شود باز هم زیاد میشود و احتمالا کروموزوم های شبیه به هم نیز خواهیم داشت و به همین دلیل ممکن است خیلی زیاد طول بکشد تا به جواب برسیم. همچنین در این حالت فضای خیلی زیادی برای اجرای الگوریتم استفاده می شود. 

2) اگر تعداد جمعیت در هر دوره افزایش پیدا کند باعث می شود دقت ما در همگرایی بیشتر شود اما زمان بیشتری برای اجرای الگوریتم طول می کشد و سرعت کاهش می یابد. 

3) تابع crossover بر روی دو کروموزوم اعمال می شود، و آنها را ترکیب میکند تا به کروموزوم هایی از ترکیب آن دو که در حالت ایده آل بهترین ویژگی های دو ژن اولیه را دارند برسد. تابع mutation بر روی یک کروموزوم اعمال میشود، و آن را جهش و یا تغییر میدهد؛ به امید آن که بتواند به کروموزوم بهتری دست یابد. در حقیقت این دو باید با هم دیگر انجام شوند زیرا اگر crossover انجام نشود ، باید تمام حالات را امتحان کنیم زیرا وقتی ترکیب نداریم کروموزوم جدیدی تولید نمی کنیم و باعث می شود خیلی طول بکشد تا به همگرایی برسیم و همچنین اگر mutation انجام نشود ، ممکن است کروموزوم ها به مسیر اشتباهی بروند و نتوانیم به جواب مساله برسیم (همگرایی را از دست بدهیم). بنابراین crossover بیشتر روی سرعت الگوریتم و mutation بیشتر روی دقت آن تاثیر می گذارند. حتی اگر هم بخواهیم فقط از یکی از این دو تکنیک استفاده کنیم crossover موثرتر از mutation خواهد بود. 

4) با تغییر دادن پارامتر های اولیه مثل جمعیت و درصد انتقال مستقیم ژن های برتر و همچنین درصد crossover می توانیم سریعتر به جواب مساله برسیم. به عنوان مثال با افزایش احتمال crossover می توانیم این کار را انجام دهیم.

5) بنطرم دلیل این اتفاق زیاد رندوم بودن در تمام مراحل این الگوریتم است. مثلا ممکن است خیلی به جواب مساله نزدیک باشیم اما به خاطر رندوم بودن یکی از این تکنیک ها به اشتباه انجام شود و باعث واگرایی شود و نگذارد همگرایی رخ دهد. مثلا تابع mutate کاملا رندوم عمل می کند و اگر بتوانیم آن را هدفمند تر اجرا کنیم تا بیشتر به همگرایی ما کمک کند سریعتر و دقیق تر به پاسخ مساله می رسیم. 

6) می توانیم یک متغیر جدا در الگوریتم تعریف کنیم که با هر بار اجرا شدن الگوریتم یکبار بشمرد و اگر در تعداد دفعات زیادی که این متغیر میشمرد ، فاصله محاسبه واقعی کروموزوم ها تا عددی که میخواهیم به آن برسیم ، ثابت مانده بود میفهمیم که مساله جواب ندارد و اجرای الگوریتم را متوقف می کنیم.